In [ ]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import os
import time
from tenacity import retry, wait_exponential, stop_after_attempt

os.environ["GROQ_API_KEY"] = "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"
prompt_text = """
You are an assistant tasked with summarizing tables and text.
Give a concise summary of the table or text.

Respond only with the summary, no additionnal comment.
Do not start your message by saying "Here is a summary" or anything like that.
Just give the summary as it is.

Table or text chunk: {element}

"""
prompt = ChatPromptTemplate.from_template(prompt_text)

# Summary chain
model = ChatGroq(temperature=0.2, model="llama3-70b-8192",max_retries=2)
summarize_chain = {"element":  RunnablePassthrough()} | prompt | model | StrOutputParser()

In [ ]:
import time

def safe_summarize(texts, summarize_chain, batch_size=2, retry_delay=15):
    summaries = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        success = False

        while not success:
            try:
                # Process batch with limited concurrency
                batch_summaries = summarize_chain.batch(batch, {"max_concurrency": 1})
                summaries.extend(batch_summaries)
                success = True
            except Exception as e:
                error_msg = str(e)
                if 'rate limit' in error_msg.lower() or '429' in error_msg:
                    print(" Rate limit hit. Retrying in", retry_delay, "seconds...")
                    time.sleep(retry_delay)
                else:
                    print(" Unexpected error:", error_msg)
                    success = True  # Exit loop even if there's an error

    return summaries


summaries = safe_summarize(texts, summarize_chain)

In [ ]:
#for image summery
'''from langchain_openai import ChatOpenAI

prompt_template = """Describe the image in detail. For context,
                  the image is part of a research paper explaining the transformers
                  architecture. Be specific about graphs, such as bar plots."""
messages = [
    (
        "user",
        [
            {"type": "text", "text": prompt_template},
            {
                "type": "image_url",
                "image_url": {"url": "data:image/jpeg;base64,{image}"},
            },
        ],
    )
]

prompt = ChatPromptTemplate.from_messages(messages)

chain = prompt | Chatgroq(model="llama3-70b-8192") | StrOutputParser()


image_summaries = chain.batch(images)'''